# NB04 — Experimentos 1/2/3 (prompting) con salida JSON estricta

Este notebook está “limpio” y organizado para:

- cargar datos (gold + memoria) sin variables fantasma
- definir prompts Exp1/Exp2/Exp3
- generar predicciones con un modelo local (Transformers) en GPU
- validar salida con **Pydantic v2** + verificación estricta de offsets/quote
- guardar resultados en JSONL

> Nota: **No** hardcodees tokens de Hugging Face aquí. Si el modelo es gated, haz login en terminal con `huggingface-cli login`.
>
> Para que no se me pierda el entorno, se puede guardar como fichero 


In [1]:
# 0) Sanity del entorno (debe ser inesagent_gpu)
import os, sys, site

print("Python:", sys.executable)
assert "/home/jovyan/.conda/envs/inesagent_gpu/bin/python" in sys.executable, "❌ No estás en el kernel inesagent_gpu"

print("PIP_USER (si existe):", os.environ.get("PIP_USER"))
print("ENABLE_USER_SITE:", site.ENABLE_USER_SITE)
print("USER_SITE:", site.getusersitepackages())

# GPU
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Si PIP_USER='yes', pip intentará instalar con --user y fallará (por el blindaje).
# Para instalar paquetes desde notebook, usaremos siempre:  env -u PIP_USER  + sys.executable -m pip


Python: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
PIP_USER (si existe): yes
ENABLE_USER_SITE: True
USER_SITE: /home/jovyan/.local/lib/python3.11/site-packages
CUDA available: True
GPU: NVIDIA A100-PCIE-40GB MIG 7g.40gb


In [2]:
# 1) Instalación opcional (solo si falta algo)
#    Ejecuta esta celda SOLO si un import falla.
import sys, importlib.util, subprocess, textwrap, os

REQUIRED = [
    "pydantic>=2",
    "jsonschema",
    "transformers>=4.45,<4.47",
    "accelerate>=0.34,<2.0",
    "huggingface_hub>=0.30,<1.0",
    "safetensors>=0.4",
    "sentencepiece",
    # cuantización 4-bit (si tu runtime lo soporta)
    "bitsandbytes>=0.43",
    # utilidades
    "tqdm",
]

def missing_pkgs(reqs):
    missing = []
    for r in reqs:
        name = r.split("==")[0].split(">=")[0].split("<")[0].strip()
        if importlib.util.find_spec(name) is None:
            missing.append(r)
    return missing

miss = missing_pkgs(REQUIRED)
print("Missing:", miss)

if miss:
    cmd = f'env -u PIP_USER "{sys.executable}" -m pip install -U --no-cache-dir ' + " ".join([repr(x) for x in miss])
    print("Running:", cmd)
    # Ejecutamos como shell para poder usar env -u PIP_USER
    r = subprocess.run(cmd, shell=True, text=True)
    if r.returncode != 0:
        raise RuntimeError("❌ pip install falló. Revisa el log arriba.")
    print("✅ Instalación terminada. Reinicia Kernel si actualizaste libs base (transformers/torch).")
else:
    print("✅ Todo instalado.")


Missing: []
✅ Todo instalado.


In [3]:
# 2) Imports (una sola vez)
from pathlib import Path
import json, re, hashlib, random
from typing import List, Dict, Any, Optional, Literal, Tuple

from pydantic import BaseModel, Field, ValidationError, field_validator

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


/home/jovyan/.conda/envs/inesagent_gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 3) Paths + utilidades de carga
#Utilidades

import re

def repair_invalid_unicode_escapes(s: str) -> str:
    return re.sub(r'\\u(?![0-9a-fA-F]{4})', r'\\\\u', s)

def normalize_quotes(s: str) -> str:
    return (s.replace("“", '"').replace("”", '"')
             .replace("’", "'").replace("‘", "'"))

from pathlib import Path
import json, hashlib

def is_jsonl(p: Path) -> bool:
    return p.suffix.lower() == ".jsonl"

def load_json(p: Path):
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(p: Path):
    rows = []
    with open(p, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

#Paths

ROOT = Path.home() / "inesagent"
assert ROOT.exists(), f"ROOT no existe: {ROOT}"
print("ROOT:", ROOT)

PATH_GOLD    = ROOT / "gold" / "corpus_annotated.jsonl"
PATH_MEMORY  = ROOT / "outputs" / "memory" / "memory_selected_CURATED.json"
PATH_BLOCKED = ROOT / "outputs" / "memory" / "blocked_keys_by_memory.json"

SPLITS_DIR = ROOT / "outputs" / "splits"
PATH_VAL   = SPLITS_DIR / "val_gold_FIXED.jsonl"
PATH_TEST  = SPLITS_DIR / "test_gold_FIXED.jsonl"
PATH_PR    = SPLITS_DIR / "prompt_regression_gold_FIXED.jsonl"
PATH_TRAIN = SPLITS_DIR / "train_gold_FIXED.jsonl" # opcional

val_gold   = load_jsonl(SPLITS_DIR / "val_gold_FIXED.jsonl")
test_gold  = load_jsonl(SPLITS_DIR / "test_gold_FIXED.jsonl")
train_gold = load_jsonl(SPLITS_DIR / "train_gold_FIXED.jsonl")
pr_gold    = load_jsonl(SPLITS_DIR / "prompt_regression_gold_FIXED.jsonl")

print("val_goldkeys:", val_gold[0].keys())

OUT_DIR = ROOT / "outputs" / "predictions"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEED = 42
random.seed(SEED)

MVP_LABELS = ["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]



for p in [PATH_GOLD, PATH_MEMORY, PATH_VAL, PATH_TEST, PATH_PR]:
    print(p, "->", p.exists())
print(PATH_TRAIN, "->", PATH_TRAIN.exists(), "(train opcional)")

blocked_obj = load_json(PATH_BLOCKED)

blocked = set()
# soporta tanto formato nuevo como legacy
if isinstance(blocked_obj, dict):
    blocked |= set(map(str, blocked_obj.get("blocked_ids", [])))
    blocked |= set(map(str, blocked_obj.get("blocked_uids", [])))
elif isinstance(blocked_obj, list):
    blocked |= set(map(str, blocked_obj))
else:
    raise ValueError("Formato de blocked inesperado")



ROOT: /home/jovyan/inesagent
val_goldkeys: dict_keys(['id', 'text', 'tags', 'legacy_doc_uid'])
/home/jovyan/inesagent/gold/corpus_annotated.jsonl -> True
/home/jovyan/inesagent/outputs/memory/memory_selected_CURATED.json -> True
/home/jovyan/inesagent/outputs/splits/val_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/test_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/prompt_regression_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/train_gold_FIXED.jsonl -> True (train opcional)


In [5]:
# 3.1) load_jsonl y load splits, comprobamos tamaños
print("train:", len(train_gold))
print("val:", len(val_gold))
print("test:", len(test_gold))
print("prompt_reg:", len(pr_gold))


train: 279
val: 34
test: 34
prompt_reg: 10


## Tamaño de muestras
- train: 279 → pool principal para construir memoria / ejemplos, o para ajuste posterior.
- val: 34 →  tamaño típico para iterar prompts y medir estabilidad sin gastar demasiado (split del corpus).
- test: 34 → mismo tamaño que val (equilibrado) (split del corpus).
- prompt_reg: 10 → conjunto pequeño para detectar regresiones de prompt (ideal que sea pequeño y fijo).

**(!)** Hacemos chequeo para comprobar y asegurarnos de que no hay solapamiento raro como:
- los ids no se repiten entre splits
- y que val/test no están dentro de blocked_ids si estás usando bloqueo por leakage.

**Respuesta esperada:**
`overlap val∩test: 0
overlap val∩train: 0
overlap test∩train: 0
overlap pr∩train: 0
overlap pr∩val: 0
overlap pr∩test: 0 (o 10)`

Que `prompt_reg ∩ test = 10` significa que ✅ prompt_reg es exactamente un subconjunto de test > que sea subconjunto de test no es un error, solo implica: No debes evaluar “test” completo y prompt_reg como si fueran dos métricas independientes, porque estarías duplicando información.

**Opción A (recomendada): deja prompt_reg como subconjunto de test**, pero úsalo bien:
- `prompt_reg`: lo usas para iterar prompts (rápido).
- `val`: lo usas para elegir el mejor prompt.
- `test`: lo usas solo al final, una vez, con el prompt congelado.

Así no hay leakage ni trampa. Opción A no tocar splits, solo metodología (prompt_reg ⊂ test). Más útil para MVP e iteración rápida

**Opción B: hacer prompt_reg independiente (si quieres “pureza”)**
- La forma más sencilla es reconstruir `prompt_reg desde train` (o desde `val`) y garantizar no solapar.
- Como ya tienes los archivos, puedes generar un nuevo `prompt_regression_gold.jsonl` (10 docs) desde train en una celda.

Opción B es más "paper-like" para evaluación

In [6]:
# 3.2) Chequeo sobre val/test vs train
def ids(docs): 
    return {d["id"] for d in docs}

ids_val = ids(val_gold)
ids_test = ids(test_gold)
ids_train = ids(train_gold)
ids_pr = ids(pr_gold)

print("overlap val∩test:", len(ids_val & ids_test))
print("overlap val∩train:", len(ids_val & ids_train))
print("overlap test∩train:", len(ids_test & ids_train))
print("overlap pr∩train:", len(ids_pr & ids_train))
print("overlap pr∩val:", len(ids_pr & ids_val))
print("overlap pr∩test:", len(ids_pr & ids_test))


overlap val∩test: 0
overlap val∩train: 0
overlap test∩train: 0
overlap pr∩train: 0
overlap pr∩val: 0
overlap pr∩test: 0


In [7]:
# 4) Cargar gold + memoria + blocked (anti-leakage)
if not PATH_GOLD.exists():
    raise FileNotFoundError(f"No encuentro gold: {PATH_GOLD}")
gold = load_jsonl(PATH_GOLD) if is_jsonl(PATH_GOLD) else load_json(PATH_GOLD)

if not PATH_MEMORY.exists():
    raise FileNotFoundError(f"No encuentro memoria: {PATH_MEMORY}")
memory_selected = load_json(PATH_MEMORY)

if not PATH_BLOCKED.exists():
    raise FileNotFoundError(f"No encuentro blocked: {PATH_BLOCKED}")
blocked = set(load_json(PATH_BLOCKED))

print("Gold docs:", len(gold))
print("Memoria ejemplos:", len(memory_selected))
print("Blocked uids:", len(blocked))


Gold docs: 373
Memoria ejemplos: 10
Blocked uids: 2


In [8]:
def doc_key(d: dict) -> str:
    """
    Clave canónica:
    - gold: id (numérico o string)
    - legacy/unannotated: doc_uid
    """
    if d.get("id") is not None:
        return str(d["id"])
    if d.get("doc_uid") is not None:
        return str(d["doc_uid"])
    return ""


In [9]:
print("PATH_MEMORY:", PATH_MEMORY)
print("PATH_BLOCKED:", PATH_BLOCKED)
print("Memory first keys:", list(memory_selected[0].keys()) if memory_selected else None)


PATH_MEMORY: /home/jovyan/inesagent/outputs/memory/memory_selected_CURATED.json
PATH_BLOCKED: /home/jovyan/inesagent/outputs/memory/blocked_keys_by_memory.json
Memory first keys: ['label', 'criterion', 'id', 'start', 'end']


In [10]:
# 5) Construir gold_mvp (solo docs que contienen alguna de las 4 etiquetas) + split val/test/train_pool
def filter_tags_mvp(tags: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return [t for t in tags if t.get("tag") in MVP_LABELS]

def has_mvp_labels_gold(doc: dict) -> bool:
    tags = doc.get("tags", [])
    if not isinstance(tags, list):
        return False
    return any(t.get("tag") in MVP_LABELS for t in tags if isinstance(t, dict))

gold_mvp = [d for d in gold if has_mvp_labels_gold(d)]
print("Gold MVP docs:", len(gold_mvp))


# pool de evaluación sin leakage
eval_pool = [d for d in gold_mvp if doc_key(d) not in blocked]
random.shuffle(eval_pool)

# tamaños robustos (si el corpus es pequeño)
test_n = min(len(eval_pool), max(30, int(0.10 * len(eval_pool))))
val_n  = min(max(0, len(eval_pool)-test_n), max(30, int(0.10 * len(eval_pool))))

gold_test = eval_pool[:test_n]
gold_val  = eval_pool[test_n:test_n+val_n]
gold_train_pool = eval_pool[test_n+val_n:]

print("Eval pool:", len(eval_pool))
print("gold_val:", len(gold_val), "gold_test:", len(gold_test), "gold_train_pool:", len(gold_train_pool))


def doc_key(d: dict) -> str:
    return str(d.get('id') or d.get('doc_uid'))


Gold MVP docs: 373
Eval pool: 373
gold_val: 37 gold_test: 37 gold_train_pool: 299


In [11]:
#Comprobamos que no sea un error que haya 373 docs con alguna de las 4 etiquetas
from collections import Counter

c = Counter()
for d in gold:
    tags = d.get("tags", [])
    labs = {t.get("tag") for t in tags if isinstance(t, dict)}
    c["has_any_mvp"] += int(bool(labs & set(MVP_LABELS)))
    c["total"] += 1
print(c)


Counter({'has_any_mvp': 373, 'total': 373})


In [12]:
# #6) Render de memoria (para Exp2/Exp3) usando texto real del gold

# 1) Índice texto por id (normalizamos a string para que coincida con memoria)
id_to_text = {str(d["id"]): d["text"] for d in gold_mvp}

def render_memory_blocks(
    memory: List[Dict[str, Any]],
    id_to_text: Dict[str, str],
    max_per_label: int = 4
) -> str:
    """
    Construye un bloque de 'memoria' para el prompt.

    Importante:
    - Tus ejemplos de memoria guardan el documento en `id` (o legacy `doc_uid`)
    - El gold guarda el documento en `id`
    - Convertimos todo a string para que las claves coincidan.
    """
    # organizar por etiqueta
    by_label: Dict[str, List[Dict[str, Any]]] = {lab: [] for lab in MVP_LABELS}
    for ex in memory:
        lab = ex.get("label")
        if lab in by_label:
            by_label[lab].append(ex)

    blocks = []
    for lab in MVP_LABELS:
        for ex in by_label[lab][:max_per_label]:
            # 2) recuperar clave del documento desde memoria
            docid = ex.get("id")
            if docid is None:
                docid = ex.get("doc_uid")  # fallback legacy

            docid = str(docid) if docid is not None else ""

            txt = id_to_text.get(docid, "")
            s, e = ex.get("start"), ex.get("end")

            # 3) defensivo: si no hay texto o offsets inválidos, marcamos placeholder
            if not txt or not isinstance(s, int) or not isinstance(e, int) or not (0 <= s < e <= len(txt)):
                span_txt = "<MISSING_TEXT_OR_BAD_OFFSETS>"
            else:
                span_txt = txt[s:e].replace("\n", " ").strip()

            blocks.append(
                f"- LABEL: {lab}\n"
                f"  CRITERION: {ex.get('criterion','')}\n"
                f"  EXAMPLE_SPAN: {span_txt}"
            )

    return "\n".join(blocks)

memory_block = render_memory_blocks(memory_selected, id_to_text, max_per_label=4)
print(memory_block[:1200], "...")



- LABEL: OBJETO
  CRITERION: P1.2 Inclusión del lote en el objeto — Se anota como “objeto” los lotes identificados que acompañan el objeto del contrato
  EXAMPLE_SPAN: SERVICIOS DE RESTAURACIÓN DE UN CONJUNTO DE 45 OBRAS PERTENECIENTES AL FONDO HISTÓRICO DE LA BIBLIOTECA DEL SENADO (LOTE 1)
- LABEL: PRECIO_DEL_CONTRATO
  CRITERION: P2.3 Precios en otros formatos — En el caso de encontrar el precio del contrato especificado como cuantía o renta periódica, desglose de precios asociados a bienes o anualidades u otro formato, se anota como “precio_del_contrato” la oración que incluye la cuantía o precio unitario junto a la periodicidad o bien al que acompaña.
  EXAMPLE_SPAN: Precio del contrato: por un importe póliza anual por persona asegurada de 3,67 €, y todo ello en los términos y condiciones que figuran en su oferta.
- LABEL: DURACION_TOTAL_DEL_CONTRATO
  CRITERION: P4 – criterio único (subtipos solo ejemplificativos)
  EXAMPLE_SPAN: El presente contrato tendrá un plazo de vigencia de

In [13]:
# 7) Few-shot extra automático para Exp3 (opcional): 1 span por etiqueta, PERO en un único JSON

import json
from typing import List, Dict, Any

def pick_one_span_per_label(pool: List[Dict[str,Any]], labels: List[str]) -> List[Dict[str,Any]]:
    """
    Devuelve una lista de items: [{"label": lab, "doc_key_id": k, "text": full_text, "span": {...}}]
    1 por etiqueta, evitando repetir doc si es posible.
    """
    out = []
    used_keys = set()

    for lab in labels:
        found = None
        for d in pool:
            k = doc_key(d)
            if k in used_keys:
                continue
            tags = d.get("tags", [])
            if any(isinstance(t, dict) and t.get("tag") == lab for t in tags):
                found = d
                break

        # fallback: si no hay doc nuevo, permitimos reutilizar doc (mejor que quedarse sin ejemplo)
        if found is None:
            for d in pool:
                tags = d.get("tags", [])
                if any(isinstance(t, dict) and t.get("tag") == lab for t in tags):
                    found = d
                    break

        if found:
            k = doc_key(found)
            used_keys.add(k)

            tag = next(t for t in found["tags"] if isinstance(t, dict) and t.get("tag") == lab)
            s, e = int(tag["start"]), int(tag["end"])
            full = found["text"]
            quote = full[s:e]

            out.append({
                "label": lab,
                "doc_key_id": str(k),
                "text": full,
                "span": {"tag": lab, "start": s, "end": e, "quote": quote}
            })

    return out


def render_fewshot_extra_one_json(fewshot_docs: List[Dict[str,Any]], context_chars: int = 350) -> str:
    """
    Renderiza UN SOLO ejemplo few-shot:
      - Construye un mini-documento concatenando ventanas alrededor de cada span
      - Devuelve UN JSON con spans (start/end relativos al mini-documento)
    """
    if not fewshot_docs:
        return ""

    pieces = []
    spans_out = []
    cursor = 0
    sep = "\n...\n"

    for ex in fewshot_docs:
        sp = ex["span"]
        full = ex["text"]
        s, e = int(sp["start"]), int(sp["end"])

        w0 = max(0, s - context_chars)
        w1 = min(len(full), e + context_chars)
        window = full[w0:w1]

        # offsets dentro de la ventana
        rs, re = s - w0, e - w0
        quote = window[rs:re]

        # coloca separador si no es la primera pieza
        if pieces:
            pieces.append(sep)
            cursor += len(sep)

        pieces.append(window)

        # offsets relativos al mini-documento concatenado
        start2 = cursor + rs
        end2   = cursor + re

        spans_out.append({
            "tag": sp["tag"],
            "start": start2,
            "end": end2,
            "quote": quote.replace("\r", "")  # opcional; si quieres cero cambios, quítalo
        })

        cursor += len(window)

    mini_text = "".join(pieces)

    # JSON único, limpio, sin escapes unicode (ensure_ascii=False)
    resp = {"spans": spans_out}
    resp_json = json.dumps(resp, ensure_ascii=False)

    block = (
        "EJEMPLO (formato correcto: UN SOLO JSON con varios spans)\n"
        "TEXTO:\n<<<TEXT>>>\n"
        f"{mini_text}\n"
        "<<<END_TEXT>>>\n"
        "RESPUESTA (JSON):\n"
        f"{resp_json}"
    )
    return block


# --- construir fewshot_extra ---
fewshot_docs = pick_one_span_per_label(gold_train_pool, MVP_LABELS)
fewshot_extra = render_fewshot_extra_one_json(fewshot_docs, context_chars=350)

print(f"fewshot_extra spans (1 por etiqueta): {len(fewshot_docs)}")
print(fewshot_extra[:8000], "...")


fewshot_extra spans (1 por etiqueta): 4
EJEMPLO (formato correcto: UN SOLO JSON con varios spans)
TEXTO:
<<<TEXT>>>
CONTRATO DE SUMINISTROS Y SERVICIOS Número de Expediente - 2023HyJ00034 SUMINISTRO EN RÉGIMEN DE ARRENDAMIENTO CON OPCIÓN A COMPRA DE LICENCIAS ILIMITADAS (ULA) DE PRODUCTOS ORACLE Y EL SOPORTE TÉCNICO ASOCIADO, ASÍ COMO LA ADQUISICIÓN DEL SERVICIO AVANZADO DE ASISTENCIA ACS REUNIDOS De una parte, Dña. Zahara De Filiberto Echeberria, en su calidad de DIRECTORA DE LA AGENCIA DE INFORMACIÓN Y CONTROL ALIMENTARIO (en adelante, AICA), en virtud de la Resolución de 7 de julio de 2019, de la Subsecretaría de Agricultura, Pesca y Alimentación, por la que se publica su nombramiento; de acuerdo con las facultades que le atribu
...
va. Además de lo anterior, el contratista se somete a la normativa nacional y de la Unión Europea en materia de protección de datos. Tercera. - Precio del contrato y revisión de precios. El precio del presente contrato, o en su caso, el importe máximo li

In [14]:
# 8) Esquema Pydantic (salida estricta) + verificación de offsets/quote
Label = Literal["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]

class Span(BaseModel):
    label: Label
    start: int = Field(ge=0)
    end: int = Field(ge=0)
    quote: str = Field(min_length=1)

    @field_validator("end")
    @classmethod
    def end_after_start(cls, v, info):
        start = info.data.get("start")
        if start is not None and v <= start:
            raise ValueError("end must be > start")
        return v

class Pred(BaseModel):
    doc_key_id: Optional[str] = None
    spans: List[Span] = Field(default_factory=list)

def strict_verify(pred: Pred, text: str) -> Pred:
    ok = []
    for sp in pred.spans:
        if sp.end > len(text):
            continue
        if text[sp.start:sp.end] != sp.quote:
            continue
        ok.append(sp)
    return Pred(doc_key_id=pred.doc_key_id, spans=ok)


In [15]:
# 9) Parseo robusto de JSON del modelo + sanitización (FIXED)
import json, re
from typing import Any, Dict, List, Tuple

def extract_all_balanced_json(text: str) -> List[str]:
    """Extrae TODOS los substrings {...} balanceados que contengan la clave 'spans'."""
    if not isinstance(text, str) or "{" not in text:
        return []
    out = []
    starts = [i for i, ch in enumerate(text) if ch == "{"]
    for s in starts:
        depth = 0
        for e in range(s, len(text)):
            ch = text[e]
            if ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    cand = text[s:e+1]
                    if '"spans"' in cand or "'spans'" in cand:
                        out.append(cand)
                    break

    # dedupe conservador (mantiene orden)
    seen = set()
    uniq = []
    for j in out:
        if j not in seen:
            seen.add(j)
            uniq.append(j)
    return uniq

def repair_invalid_unicode_escapes(s: str) -> str:
    """
    Solo repara \\uXXXX mal formados para que json.loads no explote.
    Convierte "\\u" NO seguido de 4 hex en "\\\\u" (literal).
    """

    return re.sub(r'\\u(?![0-9a-fA-F]{4})', r'\\\\u', s)

def fill_offsets_from_quote(spans: List[Dict[str,Any]], text: str) -> List[Dict[str,Any]]:
    """
    Rellena start/end buscando quote dentro de text.
    Usa cursor para encontrar ocurrencias en orden y evitar duplicados triviales.
    """
    out = []
    cursor = 0
    for sp in spans:
        tag = sp.get("tag") or sp.get("label")
        quote = sp.get("quote")

        if tag not in MVP_LABELS:
            continue
        if not isinstance(quote, str) or not quote.strip():
            continue

        idx = text.find(quote, cursor)
        if idx == -1:
            idx = text.find(quote)
        if idx == -1:
            continue

        start = idx
        end = idx + len(quote)
        out.append({"tag": tag, "start": start, "end": end, "quote": quote})
        cursor = end
    return out

def sanitize_pred_dict(obj: Dict[str,Any], doc_key_id: str, text: str) -> Dict[str,Any]:
    """
    - Filtra etiquetas no permitidas
    - Mantiene start/end si vienen numéricos (pero NO los fuerza a ser correctos aquí)
    - Rellena offsets faltantes con fill_offsets_from_quote
    """
    spans = obj.get("spans", [])
    if not isinstance(spans, list):
        spans = []

    cleaned: List[Dict[str,Any]] = []
    for sp in spans:
        if not isinstance(sp, dict):
            continue

        tag = sp.get("tag") or sp.get("label")
        quote = sp.get("quote")

        if tag not in MVP_LABELS:
            continue
        if not isinstance(quote, str) or not quote.strip():
            continue

        start = sp.get("start", None)
        end = sp.get("end", None)

        if start is not None and end is not None:
            try:
                start = int(start); end = int(end)
            except Exception:
                start = None; end = None

        cleaned.append({"tag": tag, "start": start, "end": end, "quote": quote})

    # Autofill offsets donde falten
    need_fill = [sp for sp in cleaned if sp["start"] is None or sp["end"] is None]
    have_ok  = [sp for sp in cleaned if sp["start"] is not None and sp["end"] is not None]

    filled = fill_offsets_from_quote(need_fill, text)

    final = have_ok + filled
    return {"doc_key_id": str(doc_key_id), "spans": final}

def parse_and_validate(raw: str, doc_key_id: str, text: str) -> Tuple[Dict[str,Any], str]:
    """
    - Soporta multi-JSON (Exp3 concatenando varios objetos)
    - Repara unicode escapes mal formados SOLO para poder parsear
    - Sanitiza + rellena offsets
    - Valida con Pydantic + strict_verify
    """
    candidates = extract_all_balanced_json(raw)
    if not candidates:
        return {"doc_key_id": str(doc_key_id), "spans": [], "_error": "non_json_output", "_raw": raw}, "non_json_output"

    merged_spans: List[Dict[str,Any]] = []
    parse_errors = 0

    for js in candidates:
        js2 = repair_invalid_unicode_escapes(js)
        try:
            obj = json.loads(js2)
        except Exception:
            parse_errors += 1
            continue

        spans = obj.get("spans", [])
        if isinstance(spans, list):
            merged_spans.extend(spans)

    if not merged_spans:
        # si había candidatos pero ninguno parseó bien
        if parse_errors > 0:
            return {
                "doc_key_id": str(doc_key_id),
                "spans": [],
                "_error": "json_parse_error",
                "_raw": raw,
                "_exception": f"{parse_errors} candidate(s) failed to parse"
            }, "json_parse_error"
        return {"doc_key_id": str(doc_key_id), "spans": [], "_error": "non_json_output", "_raw": raw}, "non_json_output"

    obj_merged = {"spans": merged_spans}
    obj2 = sanitize_pred_dict(obj_merged, doc_key_id, text)

    # Si tras sanitización/fill no queda nada => all_spans_discarded
    if len(obj2.get("spans", [])) == 0:
        out = {"doc_key_id": str(doc_key_id), "spans": [], "_error": "all_spans_discarded", "_raw": raw}
        out["_meta"] = {"n_spans_raw_total": len(merged_spans), "n_spans_after_fill": 0, "n_spans_final": 0}
        return out, "all_spans_discarded"

    # Pydantic + strict_verify
    try:
        pred = Pred.model_validate({
            "doc_uid": str(doc_key_id),  # Pred espera doc_uid; lo reutilizamos sin cambiar el modelo
            "spans": [
                {"label": sp["tag"], "start": sp["start"], "end": sp["end"], "quote": sp["quote"]}
                for sp in obj2["spans"]
            ]
        })
        pred2 = strict_verify(pred, text)
        out = {"doc_key_id": str(doc_key_id), "spans": [s.model_dump() for s in pred2.spans]}

        if len(out["spans"]) == 0 and len(obj2["spans"]) > 0:
            out["_error"] = "all_spans_discarded"
            out["_raw"] = raw
            out["_meta"] = {"n_spans_after_fill": len(obj2["spans"]), "n_spans_final": 0}
            return out, "all_spans_discarded"

        return out, ""

    except Exception as e:
        return {"doc_key_id": str(doc_key_id), "spans": [], "_error": "validation_error", "_raw": raw, "_exception": repr(e)}, "validation_error"


In [16]:
# 11) Carga del modelo (Transformers) — 4-bit si bitsandbytes está disponible
# Si el modelo es gated (Llama), haz login en terminal:  huggingface-cli login
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

if not torch.cuda.is_available():
    raise RuntimeError("❌ CUDA no disponible. Este notebook asume GPU.")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Intento 4-bit (bnb). Si falla, cae a fp16 (más pesado).
use_4bit = True
bnb_config = None
if use_4bit:
    try:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        print("✅ BitsAndBytesConfig OK (4-bit)")
    except Exception as e:
        print("⚠️ No puedo configurar 4-bit. Fallback fp16. Error:", repr(e))
        bnb_config = None

model_kwargs = dict(
    device_map={"": 0},
    torch_dtype=torch.float16,
)
if bnb_config is not None:
    model_kwargs["quantization_config"] = bnb_config

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **model_kwargs)
model.eval()

print("✅ Modelo cargado. device:", model.device)


✅ BitsAndBytesConfig OK (4-bit)


Loading checkpoint shards: 100%|██████████| 4/4 [02:35<00:00, 39.00s/it]


✅ Modelo cargado. device: cuda:0


In [17]:
#11.1 Celda de diagnóstico
print("tokenizer.eos_token:", tokenizer.eos_token)
print("tokenizer.eos_token_id:", tokenizer.eos_token_id)
print("tokenizer.pad_token:", tokenizer.pad_token)
print("tokenizer.pad_token_id:", tokenizer.pad_token_id)

print("model.config.eos_token_id:", model.config.eos_token_id)
print("model.config.pad_token_id:", model.config.pad_token_id)

# transformers usa a menudo generation_config en lugar de config
print("model.generation_config.eos_token_id:", getattr(model.generation_config, "eos_token_id", None))
print("model.generation_config.pad_token_id:", getattr(model.generation_config, "pad_token_id", None))


tokenizer.eos_token: <|eot_id|>
tokenizer.eos_token_id: 128009
tokenizer.pad_token: <|eot_id|>
tokenizer.pad_token_id: 128009
model.config.eos_token_id: [128001, 128008, 128009]
model.config.pad_token_id: None
model.generation_config.eos_token_id: [128001, 128008, 128009]
model.generation_config.pad_token_id: None


In [18]:
#11.2 Evitar mensajes de transformers diciendo que como el token no tiene pad definido, usa EOS para padding
# Quita warnings y define padding consistentemente
# --- FIX robusto de EOS/PAD para quitar warnings de padding ---

# 1) Determinar eos_token_id "real"
eos_id = tokenizer.eos_token_id

# Si tokenizer no lo tiene, intenta sacarlo del modelo
if eos_id is None:
    eos_id = model.config.eos_token_id

# Si eos_id es lista/tupla, coge el primero
if isinstance(eos_id, (list, tuple)):
    eos_id = eos_id[0]

# 2) Si sigue siendo None, algo está realmente mal con el tokenizer/model
assert eos_id is not None, "ERROR: eos_token_id es None. El tokenizer/model no tiene EOS configurado."

# 3) Asegura PAD en tokenizer
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = eos_id
    # opcional: token “pad” como string también
    if tokenizer.pad_token is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token

# 4) Asegura PAD/EOS en generation_config (lo que usa generate)
model.generation_config.eos_token_id = eos_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# 5) También en config, por coherencia
model.config.eos_token_id = eos_id
model.config.pad_token_id = tokenizer.pad_token_id

print("OK eos_id:", eos_id, "| pad_id:", tokenizer.pad_token_id)



OK eos_id: 128009 | pad_id: 128009


In [19]:
# Añadimos alineación de quotes a offsets 
def align_quotes_to_offsets(spans: List[Dict[str,Any]], text: str) -> List[Dict[str,Any]]:
    out = []
    for sp in spans:
        lab = sp.get("label") or sp.get("tag")
        q = sp.get("quote")
        if lab not in MVP_LABELS or not isinstance(q, str) or not q.strip():
            continue

        start = text.find(q)
        if start == -1:
            continue
        end = start + len(q)
        out.append({"label": lab, "start": start, "end": end, "quote": q})
    return out


In [20]:
# 10) Prompts Exp1/Exp2/Exp3 (MVP 4 etiquetas + offsets estrictos)
LABELS = MVP_LABELS  # ["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]
LABELS_TXT = ", ".join(LABELS)

SYSTEM_BASE = (
    "Eres un anotador jurídico experto en contratos del sector público en España.\n"
    "Tu tarea es EXTRAER FRAGMENTOS (quotes) del texto y etiquetarlos.\n\n"

    f"ETIQUETAS PERMITIDAS (OBLIGATORIO): {LABELS_TXT}.\n"
    "PROHIBIDO inventar otras etiquetas.\n\n"
    
    "IMPORTANTE: NO escapes Unicode. No uses secuencias ni escapes como \\u00fa. Escribe utf-8 directamente (á, é, í, ó, ú, ñ, €) como 'público', 'contratación', etc.\n"

    "FORMATO DE SALIDA (OBLIGATORIO):\n"
    "- Devuelve EXACTAMENTE un JSON válido y NADA más.\n"
    "- NO uses markdown (sin ```).\n"
    "- Cada span debe tener EXACTAMENTE: tag, quote.\n"
    "- Si no hay spans: {\"spans\": []}\n\n"

    "REGLAS CRÍTICAS:\n"
    "- quote debe ser un substring EXACTO del texto proporcionado (copiar/pegar del texto).\n"
    "- No inventes texto. No reformules. No normalices.\n"
    "- No incluyas <<<TEXT>>> ni <<<END_TEXT>>> dentro de quote.\n\n"

    "LÍMITES:\n"
    "- Devuelve como máximo 6 spans en total.\n"
    "- No repitas spans.\n"
)

def build_user_exp1(text: str) -> str:
    return (
        "Extrae spans SOLO para estas etiquetas: OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION.\n"
        "Devuelve SOLO JSON válido con clave 'spans'.\n"
        "Cada span debe tener: tag, quote.\n\n"
        "TEXTO:\n<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
    )

def build_user_exp2(text: str, memory_block: str) -> str:
    return (
        "Usa la MEMORIA como guía. Extrae spans SOLO para estas etiquetas: OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION.\n"
        "Devuelve SOLO JSON válido con clave 'spans'.\n"
        "Cada span debe tener: tag, quote.\n\n"
        "MEMORIA:\n"
        f"{memory_block}\n\n"
        "TEXTO:\n<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
    )

def build_user_exp3(text: str, memory_block: str, fewshot_extra: str) -> str:
    return (
        "Usa MEMORIA + FEW-SHOT EXTRA como guía. Extrae spans SOLO para estas etiquetas: OBJETO, PRECIO_DEL_CONTRATO, DURACION_TOTAL_DEL_CONTRATO, RESOLUCION.\n"
        "Devuelve SOLO JSON válido con clave 'spans'.\n"
        "Cada span debe tener: tag, quote.\n\n"
        "MEMORIA:\n"
        f"{memory_block}\n\n"
        "FEW-SHOT EXTRA:\n"
        f"{fewshot_extra}\n\n"
        "TEXTO:\n<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
    )


In [21]:
# 12) Generación chat (sin warnings) + predictor seguro
@torch.no_grad()
def generate_chat(system: str, user: str, max_new_tokens: int = 900-1200, temperature: float = 0.0, top_p: float = 0.9) -> str:
    messages = [{"role":"system","content":system}, {"role":"user","content":user}]
    enc = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", return_dict=True)
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc.get("attention_mask")
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    gen_kwargs = dict(
    max_new_tokens=512,
    do_sample=False,
    temperature=None, #no necesitamos temperatura ni top_p asi que podríamos omitirlo para evitar los warnings
    top_p=None, #no necesitamos temperatura ni top_p asi que podríamos omitirlo para evitar los warnings
    )
    
    if temperature and temperature > 0: 
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    out = model.generate(input_ids=input_ids, attention_mask=attention_mask, **gen_kwargs)
    gen_ids = out[0, input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

def predict_one(doc_id: str, text: str, exp: int) -> Dict[str,Any]:
    if exp == 1:
        user = build_user_exp1(text)
    elif exp == 2:
        user = build_user_exp2(text, memory_block)
    elif exp == 3:
        user = build_user_exp3(text, memory_block, fewshot_extra)
    else:
        raise ValueError("exp debe ser 1,2,3")

    raw = generate_chat(SYSTEM_BASE, user, max_new_tokens=900, temperature=0.0)
    pred, err = parse_and_validate(raw, doc_id, text)

# Solo por conveniencia: si hubo error y parse_and_validate no guardó raw (por cambios futuros), lo añadimos.
    if err and "_raw" not in pred:
        pred["_raw"] = raw

    return pred


## Instrucciones del prompt y experimentos

En `predict_one`:
- Exp1 llama: `build_user_exp1(text)` → solo “instrucciones + texto” (system + user), sin memoria ni ejemplos extra.
- Exp2 llama: `build_user_exp2(text, memory_block)` → instrucciones + memoria + texto
- Exp3 llama: `build_user_exp3(text, memory_block, fewshot_extra)` → instrucciones + memoria + few-shot + texto

Y en todos los casos el system es SYSTEM_BASE.

In [22]:
# 13.0.0. Definimos run_experiment + save_jsonl + rutas de salida (esto se ejecuta una vez), después se ejecutan los experimentos

from collections import Counter
from typing import List, Dict, Any, Optional
from pathlib import Path

def run_experiment(docs: List[Dict[str,Any]], exp: int, name: str, n_limit: Optional[int]=None) -> List[Dict[str,Any]]:
    out = []
    counter = Counter()
    docs2 = docs if n_limit is None else docs[:n_limit]
    for i,d in enumerate(docs2, start=1):
        k = doc_key(d)  # <- usa la función canónica
        pred = predict_one(k, d["text"], exp=exp)  # <- key, no doc_uid
        if "_error" in pred:
            counter[pred["_error"]] += 1
        out.append(pred)
        if i % 5 == 0:
            print(f"{name}: {i}/{len(docs2)} | errors:", dict(counter))
    print("DONE", name, "| total:", len(out), "| errors:", dict(counter))
    return out


def _jsonable(x):
    if isinstance(x, BaseException):
        return repr(x)
    if isinstance(x, (set, tuple)):
        return list(x)
    return x

def save_jsonl(rows: List[Dict[str,Any]], path: Path):
    import json
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False, default=_jsonable) + "\n")

OUT1 = OUT_DIR / "exp1_gold_val.jsonl"
OUT2 = OUT_DIR / "exp2_gold_val.jsonl"
OUT3 = OUT_DIR / "exp3_gold_val.jsonl"


In [23]:
# 13.0) Ejecutamos solo Exp1 (quick test)
#Como estamos ajustando prompts/parsers: usa n_limit=5 para pruebas. Cuando vaya bien, subimos a 20/35

pred1 = run_experiment(gold_val, exp=1, name="EXP1", n_limit=5)
save_jsonl(pred1, OUT1)
print("Saved:", OUT1)


EXP1: 5/5 | errors: {}
DONE EXP1 | total: 5 | errors: {}
Saved: /home/jovyan/inesagent/outputs/predictions/exp1_gold_val.jsonl


In [24]:
# 13.0.1) Ejecutamos solo Exp2 (quick test) con 5 muestras
pred2 = run_experiment(gold_val, exp=2, name="EXP2", n_limit=5)
save_jsonl(pred2, OUT2)
print("Saved:", OUT2)


KeyboardInterrupt: 

In [25]:
#b. Ver el origen de all_spans_discarded de forma agregada para saber si el mismatch es siempre el mismo patrón (saltos de línea, espacios, comillas, etc.)
#Si n_spans_kept es >0, confirma que el problema está solo en strict_verify

from collections import Counter

errs = Counter([p.get("_error","") for p in pred1 if p.get("_error")])
print(errs)

metas = [p.get("_meta", {}) for p in pred1 if p.get("_error") == "all_spans_discarded"]
print("n_spans_raw (ejemplos):", [m.get("n_spans_raw") for m in metas[:10]])
print("n_spans_kept (ejemplos):", [m.get("n_spans_kept") for m in metas[:10]])



Counter()
n_spans_raw (ejemplos): []
n_spans_kept (ejemplos): []


In [26]:
# 13) Ejecutar Exp1/Exp2/Exp3 sobre gold_val (rápido) y guardar JSONL con 5 muestras. Ajustar muestras: n_limit=20/35/50

pred1 = run_experiment(gold_val, exp=1, name="EXP1", n_limit=5)
pred2 = run_experiment(gold_val, exp=2, name="EXP2", n_limit=5)
pred3 = run_experiment(gold_val, exp=3, name="EXP3", n_limit=5)

save_jsonl(pred1, OUT1)
save_jsonl(pred2, OUT2)
save_jsonl(pred3, OUT3)

print("Saved:", OUT1, OUT2, OUT3)


EXP1: 5/5 | errors: {}
DONE EXP1 | total: 5 | errors: {}
EXP2: 5/5 | errors: {}
DONE EXP2 | total: 5 | errors: {}
EXP3: 5/5 | errors: {'all_spans_discarded': 5}
DONE EXP3 | total: 5 | errors: {'all_spans_discarded': 5}
Saved: /home/jovyan/inesagent/outputs/predictions/exp1_gold_val.jsonl /home/jovyan/inesagent/outputs/predictions/exp2_gold_val.jsonl /home/jovyan/inesagent/outputs/predictions/exp3_gold_val.jsonl


## Errors: {'all_spans_discarded'}
`EXP3: 5/5 | errors: {'all_spans_discarded': 5}`
`DONE EXP3 | total: 5 | errors: {'all_spans_discarded': 5}`

en Exp3 el modelo está “copiando” el patrón del few-shot y devolviendo offsets plantilla (0–275, 1045–1153, 1345–1458, 2465–…) que no corresponden al texto real. Por eso `strict_verify` compara `text[start:end]` vs `quote` y no coincide en ninguno, y acabas en `all_spans_discarded`.

**Por qué Exp1/Exp2 sí y Exp3 no**
- Exp1/Exp2: le pediste `tag+quote` y rellenas offsets buscando el quote en el texto → suele funcionar.
- Exp3: le metes few-shot con offsets (o con “ventanas” con offsets relativos) y el modelo aprende una “estructura” fija y repite números (36–275, 1045–1153…) aunque el texto cambie.

Fíjate: en tus 5 docs Exp3 siempre usa casi los mismos rangos. Eso NO puede ser real.

**¿De dónde salían esos offsets “1045/1345/2465…”?** De tu few-shot extra: en algún momento has generado ejemplos con:

- texto recortado/ventana
- offsets relativos al window
- y/o valores “de muestra” (36, 275, 500, 724…)

El modelo está imitando el formato y recicla números.
- Solución práctica: en Exp3 ignora los start/end del modelo
- Para Exp3 tienes que forzar el mismo comportamiento que en Exp1: usar SOLO quote y recalcular offsets siempre.

**Parche mínimo (sin rehacer nada): “modo quote-only”**
- En `parse_and_validate`, después de parsear/mergear, añade esto:
    - si `exp==3`: borra start/end de todos los spans antes de `sanitize_pred_dict` (o dentro).
    - o más general: si `strict_verify` deja 0 spans, reintenta recalculando offsets desde quote para TODOS.

Como ahora tu `parse_and_validate` no recibe `exp`, la opción más limpia es la segunda: reintento automático si `strict_verify` deja 0. Añadimos este “rescate” dentro de `parse_and_validate` (después de `strict_verify`)

## Errores con offsets del modelo

Sabiendo que los resultados del debug fueron, para el caso [1]:

    doc_uid: e65775141ab5c82bd0bd1f89e4873090c43a9569
    error: all_spans_discarded
    meta: {'n_spans_raw': 4, 'n_spans_kept': 4, 'n_spans_final': 0}

    RAW (inicio):
     {"spans": [{"tag": "OBJETO", "start": 143, "end": 153, "quote": "Obras de sustituci\u00f3n del sistema de climatizaci\u00f3n en el auditorio del edificio de Biolog\u00eda de la Facultad de Ciencias de la Universidad Aut\u00f3noma de Madrid"}, 

y

    [1] label=OBJETO start=143 end=153
    quote: 'Obras de sustitución del sistema de climatización en el auditorio del edificio de Biología de la Facultad de Ciencias de la Universidad Autónoma de Madrid'
    text[start:end]: 'bre y repr' **(!!)**
    match: False

Podemos decir que el modelo te está dando offsets (`143`…) como si el texto empezara en “Obras…”, pero en tu `text` real esa frase está en `1573`.
Eso pasa cuando el modelo no está calculando offsets sobre el texto completo, sino sobre una versión recortada o distinta del texto (p.ej. solo la sección de “CLÁUSULAS” o un extracto), o simplemente inventa offsets (muy común).

Como ya confirmaste que `find()` da `1573`, la solución práctica es:
1) Dejar de creer los offsets del modelo
Tu pipeline debe tratar start/end del modelo como “sugerencias” y hacer esto:
- usar el quote como verdad
- recalcular offsets en el texto real con find(quote)
- y solo aceptar si el quote aparece (idealmente una sola vez)

Esto exactamente lo que acabas de implementar con el `strict_verify` reparador. Con ese cambio, el span OBJETO quedará:
- `quote = “…Obras de sustitución…”`
- `pos = 1573`
- `start=1573, end=1573+len(quote)`
- y ya no habrá `all_spans_discarded`.

2) ¿Por qué el modelo devuelve `143` entonces?
Puede ser cualquiera de estas (todas típicas):
    A) El modelo “no sabe” calcular offsets globales
        - Muchísimos LLMs fallan con offsets en textos largos. A veces ponen números pequeños “porque sí”.
    B) Tu prompt/plantilla induce al modelo a pensar que el texto empieza en otro punto
        - Si el modelo se fija en una sección posterior (“CLÁUSULAS CONTRACTUALES…”) puede tomar esa como inicio mental.
    C) Texto muy largo + atención limitada

Aunque le metas todo el texto, puede “anclar” el cálculo de offsets a lo que tiene más cerca en contexto.

Unicode NO: en tu caso el quote está como \u00f3 pero eso al parsear vuelve a “ó”. No cambia la longitud del string original de tu text, y además el mismatch es de 143 vs 1573 (enorme), no de 1–2 chars.

**(*) Opción recomendada (robusta y simple)**: Cambiar el contrato de salida:
- que el modelo NO devuelva offsets, solo label + quote, y yo calculo offsets siempre haciendo `find()` y añadiendo `start/end`.


**La causa más probable (99%): normalización de texto**
`strict_verify` suele fallar por:
- `\r\n` vs `\n`
- espacios múltiples vs uno
- comillas tipográficas “ ” vs "
- guiones largos – vs -
- OCR con caracteres raros
- el modelo devolviendo quote sin exactamente las mismas nuevas líneas

Tu verificación es “exact string match” y es demasiado estricta para texto con OCR/ruido.

Hacemos un arreglo (estratégicamente):

**Estrategia 1**: si quote no coincide, buscarlo literalmente en el texto y corregir offsets
- Si el modelo te da quote, intenta text.find(quote).
- Si aparece una sola vez, reemplaza start/end por esa posición.
- Si aparece varias veces, o no aparece, descarta.

Esto mantiene precisión (solo aceptamos quotes que están realmente en el texto), pero no dependemos de offsets del modelo.

## ¿Los offsets son correctos con respecto al texto?

✅ Sí, si pasaron por `strict_verify()` (y/o por `fill_offsets_from_quote()` + `verify`).
Esto es validación dura.

## ¿Los offsets son correctos con respecto a las anotaciones GOLD (mismo span que gold)




In [27]:
#Checks de offsets contra texto (100% determinista)
def check_offsets_vs_text(pred_rows, gold_val):
    # index por id
    id_to_text = {str(doc_key(d)): d["text"] for d in gold_val}

    ok = 0
    bad = 0
    for r in pred_rows:
        k = str(r["doc_key_id"])
        t = id_to_text.get(k)
        if t is None:
            print("MISSING TEXT FOR:", k)
            bad += 1
            continue

        for sp in r.get("spans", []):
            s, e, q = sp["start"], sp["end"], sp["quote"]
            if t[s:e] != q:
                bad += 1
                print("\nBAD:", k, sp["label"], (s,e))
                print("expected:", repr(t[s:e][:120]))
                print("got     :", repr(q[:120]))
            else:
                ok += 1
    print("OK spans:", ok, "| BAD spans:", bad)
check_offsets_vs_text(pred1, gold_val[:5])
check_offsets_vs_text(pred2, gold_val[:5])
check_offsets_vs_text(pred3, gold_val[:5])


OK spans: 20 | BAD spans: 0
OK spans: 16 | BAD spans: 0
OK spans: 0 | BAD spans: 0


**Si te da BAD spans: 0, offsets perfectos.**

In [28]:
# Check contra gold tags (match con anotación)
# Primero necesitas mapear el doc del split al doc del gold completo (por id).
# En tu gold real los tags son tipo: {"start":..., "end":..., "tag":"OBJETO", "token_start":..., "token_end":...}

# gold_full: lista de docs gold cargados de PATH_GOLD (con keys: id, text, tags)
gold_full = gold
gold_by_id = {str(d["id"]): d for d in gold_full}

def overlap(a_start, a_end, b_start, b_end):
    return max(0, min(a_end,b_end) - max(a_start,b_start))

def check_against_gold(pred_rows, gold_full):
    gold_by_id = {str(d["id"]): d for d in gold_full}
    for r in pred_rows:
        k = str(r["doc_key_id"])
        g = gold_by_id.get(k)
        if not g:
            print("No gold for:", k)
            continue

        gold_tags = [t for t in g.get("tags", []) if t.get("tag") in MVP_LABELS]
        print("\nDOC", k, "| gold tags:", len(gold_tags), "| pred spans:", len(r.get("spans", [])))

        for sp in r.get("spans", []):
            lab = sp["label"]
            s,e = sp["start"], sp["end"]

            candidates = [t for t in gold_tags if t.get("tag")==lab]
            if not candidates:
                print("  pred", lab, "-> gold has none")
                continue

            # mira mejor solape
            best = max(overlap(s,e, t["start"], t["end"]) for t in candidates)
            print(f"  {lab}: best overlap chars = {best}")

check_against_gold(pred1, gold_full=gold)  # 'gold' = tu gold cargado de PATH_GOLD




DOC 416404459 | gold tags: 3 | pred spans: 4
  OBJETO: best overlap chars = 45
  PRECIO_DEL_CONTRATO: best overlap chars = 12
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 11
  pred RESOLUCION -> gold has none

DOC 588320436 | gold tags: 6 | pred spans: 4
  OBJETO: best overlap chars = 75
  PRECIO_DEL_CONTRATO: best overlap chars = 26
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 56
  pred RESOLUCION -> gold has none

DOC 81854174 | gold tags: 4 | pred spans: 4
  OBJETO: best overlap chars = 63
  PRECIO_DEL_CONTRATO: best overlap chars = 14
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 57
  pred RESOLUCION -> gold has none

DOC 2132006064 | gold tags: 6 | pred spans: 4
  OBJETO: best overlap chars = 186
  PRECIO_DEL_CONTRATO: best overlap chars = 84
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 105
  RESOLUCION: best overlap chars = 128

DOC 954871603 | gold tags: 5 | pred spans: 4
  OBJETO: best overlap chars = 72
  PRECIO_DEL_CONTRATO: best overlap chars = 15

In [29]:
# Check pred2 contra gold tags (match con anotación)
# Primero necesitas mapear el doc del split al doc del gold completo (por id).
# En tu gold real los tags son tipo: {"start":..., "end":..., "tag":"OBJETO", "token_start":..., "token_end":...}

# gold_full: lista de docs gold cargados de PATH_GOLD (con keys: id, text, tags)
gold_full = gold
gold_by_id = {str(d["id"]): d for d in gold_full}

def overlap(a_start, a_end, b_start, b_end):
    return max(0, min(a_end,b_end) - max(a_start,b_start))

def check_against_gold(pred_rows, gold_full):
    gold_by_id = {str(d["id"]): d for d in gold_full}
    for r in pred_rows:
        k = str(r["doc_key_id"])
        g = gold_by_id.get(k)
        if not g:
            print("No gold for:", k)
            continue

        gold_tags = [t for t in g.get("tags", []) if t.get("tag") in MVP_LABELS]
        print("\nDOC", k, "| gold tags:", len(gold_tags), "| pred spans:", len(r.get("spans", [])))

        for sp in r.get("spans", []):
            lab = sp["label"]
            s,e = sp["start"], sp["end"]

            candidates = [t for t in gold_tags if t.get("tag")==lab]
            if not candidates:
                print("  pred", lab, "-> gold has none")
                continue

            # mira mejor solape
            best = max(overlap(s,e, t["start"], t["end"]) for t in candidates)
            print(f"  {lab}: best overlap chars = {best}")

check_against_gold(pred2, gold_full=gold)  # 'gold' = tu gold cargado de PATH_GOLD




DOC 416404459 | gold tags: 3 | pred spans: 1
  PRECIO_DEL_CONTRATO: best overlap chars = 12

DOC 588320436 | gold tags: 6 | pred spans: 4
  OBJETO: best overlap chars = 75
  PRECIO_DEL_CONTRATO: best overlap chars = 26
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 56
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 0

DOC 81854174 | gold tags: 4 | pred spans: 4
  OBJETO: best overlap chars = 63
  PRECIO_DEL_CONTRATO: best overlap chars = 14
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 57
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 0

DOC 2132006064 | gold tags: 6 | pred spans: 3
  OBJETO: best overlap chars = 186
  RESOLUCION: best overlap chars = 128
  RESOLUCION: best overlap chars = 0

DOC 954871603 | gold tags: 5 | pred spans: 4
  OBJETO: best overlap chars = 72
  PRECIO_DEL_CONTRATO: best overlap chars = 15
  DURACION_TOTAL_DEL_CONTRATO: best overlap chars = 270
  RESOLUCION: best overlap chars = 172


In [31]:
# Check pred3 contra gold tags (match con anotación)
# Primero necesitas mapear el doc del split al doc del gold completo (por id).
# En tu gold real los tags son tipo: {"start":..., "end":..., "tag":"OBJETO", "token_start":..., "token_end":...}

# gold_full: lista de docs gold cargados de PATH_GOLD (con keys: id, text, tags)
gold_full = gold
gold_by_id = {str(d["id"]): d for d in gold_full}

def overlap(a_start, a_end, b_start, b_end):
    return max(0, min(a_end,b_end) - max(a_start,b_start))

def check_against_gold(pred_rows, gold_full):
    gold_by_id = {str(d["id"]): d for d in gold_full}
    for r in pred_rows:
        k = str(r["doc_key_id"])
        g = gold_by_id.get(k)
        if not g:
            print("No gold for:", k)
            continue

        gold_tags = [t for t in g.get("tags", []) if t.get("tag") in MVP_LABELS]
        print("\nDOC", k, "| gold tags:", len(gold_tags), "| pred spans:", len(r.get("spans", [])))

        for sp in r.get("spans", []):
            lab = sp["label"]
            s,e = sp["start"], sp["end"]

            candidates = [t for t in gold_tags if t.get("tag")==lab]
            if not candidates:
                print("  pred", lab, "-> gold has none")
                continue

            # mira mejor solape
            best = max(overlap(s,e, t["start"], t["end"]) for t in candidates)
            print(f"  {lab}: best overlap chars = {best}")

check_against_gold(pred3, gold_full=gold)  # 'gold' = tu gold cargado de PATH_GOLD




DOC 416404459 | gold tags: 3 | pred spans: 0

DOC 588320436 | gold tags: 6 | pred spans: 0

DOC 81854174 | gold tags: 4 | pred spans: 0

DOC 2132006064 | gold tags: 6 | pred spans: 0

DOC 954871603 | gold tags: 5 | pred spans: 0


**Interpretación rápida:**
- best overlap chars = 0 → el modelo eligió un sitio distinto al gold para esa etiqueta.
- overlap grande pero no total → el modelo eligió un subspan o superspan.
- overlap cercano al largo → casi coincide.

In [30]:
# 14) Supervisor: inspección rápida de outputs guardados
def read_jsonl(path: Path, n: int = 3):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i,line in enumerate(f):
            if i >= n: break
            rows.append(json.loads(line))
    return rows

for path in [OUT1, OUT2, OUT3]:
    print("\n===", path.name, "===")
    rows = read_jsonl(path, n=2)
    for r in rows:
        print("doc_key_id:", r.get("doc_key_id"), "| spans:", len(r.get("spans",[])), "| error:", r.get("_error"))
        for sp in r.get("spans", [])[:3]:
            print(" ", sp["label"], sp["start"], sp["end"], "| quote[:80]=", sp["quote"][:80].replace("\n"," "))



=== exp1_gold_val.jsonl ===
doc_key_id: 416404459 | spans: 4 | error: None
  OBJETO 243 288 | quote[:80]= VIGILANCIA DE LA SALUD (MEDICINA DEL TRABAJO)
  PRECIO_DEL_CONTRATO 2216 2290 | quote[:80]= 377.772,00 €, más el correspondiente IVA (6.869,52 €), Total: 384.641,52 €
  DURACION_TOTAL_DEL_CONTRATO 2363 2374 | quote[:80]= CUATRO años
doc_key_id: 588320436 | spans: 4 | error: None
  OBJETO 2273 2461 | quote[:80]= Suministro de repuesto y accesorios para los sistemas de armas de aeronaves De a
  PRECIO_DEL_CONTRATO 3355 3408 | quote[:80]= CIENTO CINCUENTA MIL EUROS, IVA EXENTO (150.000,00 €)
  DURACION_TOTAL_DEL_CONTRATO 3717 3773 | quote[:80]= desde su formalización hasta el 30 de noviembre de 2024.

=== exp2_gold_val.jsonl ===
doc_key_id: 416404459 | spans: 1 | error: None
  PRECIO_DEL_CONTRATO 2178 2304 | quote[:80]= El PRECIO del presente contrato es de 377.772,00 €, más el correspondiente IVA (
doc_key_id: 588320436 | spans: 4 | error: None
  OBJETO 2273 2461 | quote[:80]= Sumin